In [1]:
from pathlib import Path
import os
import torch
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import datetime
from torch import nn

In [2]:
# Téléchargement des données

from datamaestro import prepare_dataset
ds = prepare_dataset("com.lecun.mnist");
train_images, train_labels = ds.train.images.data(), ds.train.labels.data()
test_images, test_labels =  ds.test.images.data(), ds.test.labels.data()

In [3]:
# Tensorboard : rappel, lancer dans une console tensorboard --logdir runs
writer = SummaryWriter("runs/runs"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# Pour visualiser
# Les images doivent etre en format Channel (3) x Hauteur x Largeur
images = torch.tensor(train_images[0:8]).unsqueeze(1).repeat(1,3,1,1).double()/255.
# Permet de fabriquer une grille d'images
images = make_grid(images)
# Affichage avec tensorboard
writer.add_image(f'samples', images, 0)


savepath = Path("model.pch")

#  TODO: 

<ipython-input-3-7eff804f32da>:6: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  images = torch.tensor(train_images[0:8]).unsqueeze(1).repeat(1,3,1,1).double()/255.


In [87]:
from torch.utils.data import Dataset, DataLoader
class MonDataset(Dataset):
    
    def __init__(self, data, target):
        self.data = torch.tensor(data.reshape((data.shape[0], data.shape[1]*data.shape[2])), dtype=torch.float)
        self.target = target
    
    def __getitem__(self, index):
        return (self.data[index], self.target[index])
    
    def __len__(self):
        return len(self.data)

In [101]:
BATCH_SIZE = 1000
data = DataLoader(MonDataset(train_images, train_labels) ,shuffle=True, batch_size=BATCH_SIZE)

In [105]:
model = torch.nn.Sequential(
          nn.Linear(784, 28, bias=True),
          nn.ReLU(),
          nn.Linear(28, 784, bias=True),
          nn.Sigmoid(),
        )

In [106]:
epsilon = 0.01

optim = torch.optim.SGD(params=model.parameters(),lr=epsilon)
optim.zero_grad()

criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()

In [107]:
for n_iter in range(30):
    
    for x_batch, y_batch in data:
        
        # Pass Forward
        yhat = model(x_batch)
        loss = criterion(yhat, x_batch)
        
        # Pass backward
        loss.backward()
        
        # Updating parameters & reset grad
        optim.step()
        optim.zero_grad()
        
    if(n_iter%10==0):
        print(f"Itérations {n_iter}: loss {loss}")

Itérations 0: loss 7187.87841796875
Itérations 10: loss 7140.73388671875
Itérations 20: loss 7115.4130859375


# Higway network

In [110]:
class higwayModule(nn.Module):
    
    def __init__(self, inputDim, outputDim):
        super(Net, self).__init__()
        
        # Transform gate
        self.linear1 = nn.Linear(inputDim, outputDim, bias=True)#output dim of T ?
        self.sigmoid = nn.Sigmoid()
        
        # Normal layer
        self.linear2 = nn.Linear(inputDim, outputDim, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        
        T = self.sigmoid(self.linear1(x))
        C = 1.0 - T
        H = self.relu(self.linear2(x))
        
        return T*H+x*C